#Stack overflow surveys analysis

In [2]:
import pandas as pd 

In [68]:
na_vals=['NA','Missing'] #if we have such values we change them to NAN 
df = pd.read_csv('survey_results_public.csv',index_col='Respondent', na_values= na_vals)
schema_df= pd.read_csv('survey_results_schema.csv')

In [4]:
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,EUR,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",White or of European descent,Man,"Languages, frameworks, and other technologies ...",Slightly satisfied,I am not interested in new job opportunities,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,.NET Core;Xamarin,.NET;.NET Core,Microsoft Teams;Microsoft Azure;Trello,Confluence;Jira;Slack;Microsoft Azure;Trello,No,Somewhat important,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,Often: 1-2 days per week or more,Start a free trial;Ask developers I know/work ...,Amused,Stack Overflow (public Q&A for anyone who codes),Visit Stack Overflow;Go for a walk or other ph...,Windows,2 to 9 employees,Android;iOS;Kubernetes;Microsoft Azure;Windows,Windows,NaN,Straight / Heterosexual,No,"No, not at all",NaN,Multiple times per day,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,GBP,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,NaN,Very dissatisfied,I am not interested in new job opportunities,Python;Swift,JavaScript;Swift,React Native;TensorFlow;Unity 3D,React Native,Github;Slack,Confluence;Jira;Github;Gitlab;Slack,NaN,NaN,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,NaN,NaN,Amused,Stack Overflow (public Q&A for anyone who code...,Visit Stack Overflow;Go for a walk or other ph...,MacOS,"1,000 to 4,999 employees",iOS;Kubernetes;Linux;MacOS,iOS,I have little or no influence,NaN,Yes,"Yes, definitely",Less than once per month or monthly,Multiple times per day,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Once a decade,NaN,NaN,No,NaN,NaN,NaN,Stack Overflow (public Q&A for anyone who codes),NaN,Linux-based,NaN,NaN,NaN,NaN,NaN,Yes,"Yes, somewhat",A few times per month or weekly,Daily or almost daily,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,ALL,NaN,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,White or of European descent,Man,Flex time or a flexible schedule;Office enviro...,Slightly dissatisfied,"I’m not actively looking, but I am open to new...",NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,Not at all important/not necessary,Curious about other opportunities;Wanting to w...,NaN,Once a year,Not sure,Yes,Yes,Occasionally: 1-2 days per quarter but less th...,NaN,NaN,Stack Overflow (public Q&A for anyone who code...,NaN,Lin

In [5]:
df.shape


(64461, 61)

In [6]:
schema_df.shape

(61, 2)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64461 entries, 0 to 64460
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Respondent                    64461 non-null  int64  
 1   MainBranch                    64162 non-null  object 
 2   Hobbyist                      64416 non-null  object 
 3   Age                           45446 non-null  float64
 4   Age1stCode                    57900 non-null  object 
 5   CompFreq                      40069 non-null  object 
 6   CompTotal                     34826 non-null  float64
 7   ConvertedComp                 34756 non-null  float64
 8   Country                       64072 non-null  object 
 9   CurrencyDesc                  45472 non-null  object 
 10  CurrencySymbol                45472 non-null  object 
 11  DatabaseDesireNextYear        44070 non-null  object 
 12  DatabaseWorkedWith            49537 non-null  object 
 13  D

In [8]:
schema_df.head()

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?
3,Age,What is your age (in years)? If you prefer not...
4,Age1stCode,At what age did you write your first line of c...


In [9]:
pd.set_option('display.max_rows',61)
#in order to visualize all the rows

In [10]:
schema_df

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?
3,Age,What is your age (in years)? If you prefer not...
4,Age1stCode,At what age did you write your first line of c...
5,CompFreq,"Is that compensation weekly, monthly, or yearly?"
6,CompTotal,What is your current total compensation (salar...
7,ConvertedComp,Salary converted to annual USD salaries using ...
8,Country,Where do you live?
9,CurrencyDesc,Which currency do you use day-to-day? If your ...


In [11]:
df.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'Age', 'Age1stCode', 'CompFreq',
       'CompTotal', 'ConvertedComp', 'Country', 'CurrencyDesc',
       'CurrencySymbol', 'DatabaseDesireNextYear', 'DatabaseWorkedWith',
       'DevType', 'EdLevel', 'Employment', 'Ethnicity', 'Gender', 'JobFactors',
       'JobSat', 'JobSeek', 'LanguageDesireNextYear', 'LanguageWorkedWith',
       'MiscTechDesireNextYear', 'MiscTechWorkedWith',
       'NEWCollabToolsDesireNextYear', 'NEWCollabToolsWorkedWith', 'NEWDevOps',
       'NEWDevOpsImpt', 'NEWEdImpt', 'NEWJobHunt', 'NEWJobHuntResearch',
       'NEWLearn', 'NEWOffTopic', 'NEWOnboardGood', 'NEWOtherComms',
       'NEWOvertime', 'NEWPurchaseResearch', 'NEWPurpleLink', 'NEWSOSites',
       'NEWStuck', 'OpSys', 'OrgSize', 'PlatformDesireNextYear',
       'PlatformWorkedWith', 'PurchaseWhat', 'Sexuality', 'SOAccount',
       'SOComm', 'SOPartFreq', 'SOVisitFreq', 'SurveyEase', 'SurveyLength',
       'Trans', 'UndergradMajor', 'WebframeDesireNextYear',
  

In [12]:
df.loc[0]
#the answers of the first person

Respondent                                                                      1
MainBranch                                         I am a developer by profession
Hobbyist                                                                      Yes
Age                                                                           NaN
Age1stCode                                                                     13
CompFreq                                                                  Monthly
CompTotal                                                                     NaN
ConvertedComp                                                                 NaN
Country                                                                   Germany
CurrencyDesc                                                        European Euro
CurrencySymbol                                                                EUR
DatabaseDesireNextYear                                       Microsoft SQL Server
DatabaseWorkedWi

In [13]:
df.iloc[0]
#accessing the row of index 0

Respondent                                                                      1
MainBranch                                         I am a developer by profession
Hobbyist                                                                      Yes
Age                                                                           NaN
Age1stCode                                                                     13
CompFreq                                                                  Monthly
CompTotal                                                                     NaN
ConvertedComp                                                                 NaN
Country                                                                   Germany
CurrencyDesc                                                        European Euro
CurrencySymbol                                                                EUR
DatabaseDesireNextYear                                       Microsoft SQL Server
DatabaseWorkedWi

In [14]:
df.iloc[[0,1],2]
#the second column of the first and the second row
#df.loc[0]
#give us the first row


0    Yes
1     No
Name: Hobbyist, dtype: object

In [15]:
df.loc[[20,22],'Hobbyist']

20    Yes
22    Yes
Name: Hobbyist, dtype: object

In [16]:
df['Hobbyist'].value_counts()

Yes    50388
No     14028
Name: Hobbyist, dtype: int64

In [17]:
df['Gender'].value_counts()

Man                                                            46013
Woman                                                           3844
Non-binary, genderqueer, or gender non-conforming                385
Man;Non-binary, genderqueer, or gender non-conforming            121
Woman;Non-binary, genderqueer, or gender non-conforming           92
Woman;Man                                                         76
Woman;Man;Non-binary, genderqueer, or gender non-conforming       26
Name: Gender, dtype: int64

In [18]:
df.loc[20:22,'Hobbyist':'Gender']

,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender
20,Yes,23.0,16,NaN,NaN,NaN,Belgium,NaN,NaN,Firebase;MariaDB;MongoDB,Firebase;MariaDB;Microsoft SQL Server;MongoDB;...,NaN,"Secondary school (e.g. American high school, G...",Student,White or of European descent,Man
21,Yes,NaN,18,Monthly,NaN,NaN,India,Indian rupee,INR,NaN,NaN,"Developer, full-stack;Engineer, data","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,Man
22,Yes,27.0,14,NaN,NaN,NaN,Germany,European Euro,EUR,NaN,NaN,"Developer, back-end;Developer, desktop or ente...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,White or of European descent,Man


In [19]:
df['Respondent'].value_counts()
#so we confirm we can use it as an index beacuse it is unique id 

2047     1
49805    1
31386    1
25241    1
27288    1
        ..
15725    1
13676    1
3435     1
1386     1
2049     1
Name: Respondent, Length: 64461, dtype: int64

In [20]:
df.set_index('Respondent')

,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
Respondent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,EUR,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",White or of European descent,Man,"Languages, frameworks, and other technologies ...",Slightly satisfied,I am not interested in new job opportunities,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,.NET Core;Xamarin,.NET;.NET Core,Microsoft Teams;Microsoft Azure;Trello,Confluence;Jira;Slack;Microsoft Azure;Trello,No,Somewhat important,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,Often: 1-2 days per week or more,Start a free trial;Ask developers I know/work ...,Amused,Stack Overflow (public Q&A for anyone who codes),Visit Stack Overflow;Go for a walk or other ph...,Windows,2 to 9 employees,Android;iOS;Kubernetes;Microsoft Azure;Windows,Windows,NaN,Straight / Heterosexual,No,"No, not at all",NaN,Multiple times per day,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,GBP,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,NaN,Very dissatisfied,I am not interested in new job opportunities,Python;Swift,JavaScript;Swift,React Native;TensorFlow;Unity 3D,React Native,Github;Slack,Confluence;Jira;Github;Gitlab;Slack,NaN,NaN,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,NaN,NaN,Amused,Stack Overflow (public Q&A for anyone who code...,Visit Stack Overflow;Go for a walk or other ph...,MacOS,"1,000 to 4,999 employees",iOS;Kubernetes;Linux;MacOS,iOS,I have little or no influence,NaN,Yes,"Yes, definitely",Less than once per month or monthly,Multiple times per day,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Once a decade,NaN,NaN,No,NaN,NaN,NaN,Stack Overflow (public Q&A for anyone who codes),NaN,Linux-based,NaN,NaN,NaN,NaN,NaN,Yes,"Yes, somewhat",A few times per month or weekly,Daily or almost daily,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,ALL,NaN,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,White or of European descent,Man,Flex time or a flexible schedule;Office enviro...,Slightly dissatisfied,"I’m not actively looking, but I am open to new...",NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,Not at all important/not necessary,Curious about other opportunities;Wanting to w...,NaN,Once a year,Not sure,Yes,Yes,Occasionally: 1-2 days per quarter but less th...,NaN,NaN,Stack

In [21]:
#in order to facilitate the search of the corresponding qustion, we will set the column :'columns' in the data frame schema_df as indexes
schema_df.set_index('Column')

,QuestionText
Column,
Respondent,Randomized respondent ID number (not in order ...
MainBranch,Which of the following options best describes ...
Hobbyist,Do you code as a hobby?
Age,What is your age (in years)? If you prefer not...
Age1stCode,At what age did you write your first line of c...
CompFreq,"Is that compensation weekly, monthly, or yearly?"
CompTotal,What is your current total compensation (salar...
ConvertedComp,Salary converted to annual USD salaries using ...
Country,Where do you live?


In [22]:
#create the filter 
high_salary =(df['ConvertedComp']>70000)

In [23]:
#apply the filter 
df.loc[high_salary,['Country','LanguageWorkedWith','ConvertedComp']]

,Country,LanguageWorkedWith,ConvertedComp
7,United States,Python;SQL,116000.0
15,United Kingdom,Bash/Shell/PowerShell;HTML/CSS;Java;JavaScript...,108576.0
16,United States,C#;HTML/CSS;JavaScript;Python;SQL;VBA,79000.0
17,United States,Bash/Shell/PowerShell;HTML/CSS;Perl,1260000.0
18,United States,Bash/Shell/PowerShell;C#;HTML/CSS;JavaScript;S...,83400.0
...,...,...,...
64113,United States,NaN,225000.0
64116,United States,NaN,150000.0
64127,United States,NaN,140000.0
64129,United States,NaN,150000.0


In [24]:
countries=['United States','India','United Kingdom','Germany','Canada']
filt= df['Country'].isin(countries)

In [25]:
df.loc[filt,['Country','ConvertedComp']]

,Country,ConvertedComp
0,Germany,NaN
1,United Kingdom,NaN
4,United States,NaN
5,Germany,NaN
6,India,NaN
...,...,...
64441,Canada,NaN
64442,India,NaN
64443,United States,NaN
64452,India,NaN


In [26]:
df.loc[filt,['Country','LanguageWorkedWith']]

,Country,LanguageWorkedWith
0,Germany,C#;HTML/CSS;JavaScript
1,United Kingdom,JavaScript;Swift
4,United States,HTML/CSS;Ruby;SQL
5,Germany,HTML/CSS;Java;JavaScript
6,India,C#;HTML/CSS;PHP
...,...,...
64441,Canada,C;C#;C++;Java
64442,India,SQL
64443,United States,C++;HTML/CSS;Java;JavaScript;Python;SQL
64452,India,NaN


In [27]:
filt= df['LanguageWorkedWith'].str.contains('Python', na=False)

In [28]:
df.loc[filt,['LanguageWorkedWith']]

,LanguageWorkedWith
2,Objective-C;Python;Swift
7,Python;SQL
9,HTML/CSS;Java;JavaScript;Python;SQL
12,C;JavaScript;Python
14,Bash/Shell/PowerShell;C;HTML/CSS;Java;Python;SQL
...,...
64433,Bash/Shell/PowerShell;HTML/CSS;JavaScript;Perl...
64438,C++;HTML/CSS;JavaScript;Python;Ruby;TypeScript
64443,C++;HTML/CSS;Java;JavaScript;Python;SQL
64446,Bash/Shell/PowerShell;C;C#;C++;HTML/CSS;Java;J...


In [29]:
#renaming the column ConvertedComp to a clearer name
df.rename(columns={'ConvertedComp':'SalaryUSD'}, inplace=True)

In [30]:
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,SalaryUSD,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,EUR,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",White or of European descent,Man,"Languages, frameworks, and other technologies ...",Slightly satisfied,I am not interested in new job opportunities,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,.NET Core;Xamarin,.NET;.NET Core,Microsoft Teams;Microsoft Azure;Trello,Confluence;Jira;Slack;Microsoft Azure;Trello,No,Somewhat important,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,Often: 1-2 days per week or more,Start a free trial;Ask developers I know/work ...,Amused,Stack Overflow (public Q&A for anyone who codes),Visit Stack Overflow;Go for a walk or other ph...,Windows,2 to 9 employees,Android;iOS;Kubernetes;Microsoft Azure;Windows,Windows,NaN,Straight / Heterosexual,No,"No, not at all",NaN,Multiple times per day,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,GBP,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,NaN,Very dissatisfied,I am not interested in new job opportunities,Python;Swift,JavaScript;Swift,React Native;TensorFlow;Unity 3D,React Native,Github;Slack,Confluence;Jira;Github;Gitlab;Slack,NaN,NaN,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,NaN,NaN,Amused,Stack Overflow (public Q&A for anyone who code...,Visit Stack Overflow;Go for a walk or other ph...,MacOS,"1,000 to 4,999 employees",iOS;Kubernetes;Linux;MacOS,iOS,I have little or no influence,NaN,Yes,"Yes, definitely",Less than once per month or monthly,Multiple times per day,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Once a decade,NaN,NaN,No,NaN,NaN,NaN,Stack Overflow (public Q&A for anyone who codes),NaN,Linux-based,NaN,NaN,NaN,NaN,NaN,Yes,"Yes, somewhat",A few times per month or weekly,Daily or almost daily,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,ALL,NaN,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,White or of European descent,Man,Flex time or a flexible schedule;Office enviro...,Slightly dissatisfied,"I’m not actively looking, but I am open to new...",NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,Not at all important/not necessary,Curious about other opportunities;Wanting to w...,NaN,Once a year,Not sure,Yes,Yes,Occasionally: 1-2 days per quarter but less th...,NaN,NaN,Stack Overflow (public Q&A for anyone who code...,NaN,Linux-b

In [31]:
df['Hobbyist']

0        Yes
1         No
2        Yes
3        Yes
4        Yes
        ... 
64456    Yes
64457    Yes
64458    Yes
64459    Yes
64460    Yes
Name: Hobbyist, Length: 64461, dtype: object

In [32]:
df['Hobbyist']=df['Hobbyist'].map({'Yes':True,'No':False})

In [33]:
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,SalaryUSD,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,True,NaN,13,Monthly,NaN,NaN,Germany,European Euro,EUR,Microsoft SQL Server,Elasticsearch;Microsoft SQL Server;Oracle,"Developer, desktop or enterprise applications;...","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",White or of European descent,Man,"Languages, frameworks, and other technologies ...",Slightly satisfied,I am not interested in new job opportunities,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript,.NET Core;Xamarin,.NET;.NET Core,Microsoft Teams;Microsoft Azure;Trello,Confluence;Jira;Slack;Microsoft Azure;Trello,No,Somewhat important,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,Often: 1-2 days per week or more,Start a free trial;Ask developers I know/work ...,Amused,Stack Overflow (public Q&A for anyone who codes),Visit Stack Overflow;Go for a walk or other ph...,Windows,2 to 9 employees,Android;iOS;Kubernetes;Microsoft Azure;Windows,Windows,NaN,Straight / Heterosexual,No,"No, not at all",NaN,Multiple times per day,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,False,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,GBP,NaN,NaN,"Developer, full-stack;Developer, mobile","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,NaN,Very dissatisfied,I am not interested in new job opportunities,Python;Swift,JavaScript;Swift,React Native;TensorFlow;Unity 3D,React Native,Github;Slack,Confluence;Jira;Github;Gitlab;Slack,NaN,NaN,Fairly important,NaN,NaN,Once a year,Not sure,NaN,No,NaN,NaN,Amused,Stack Overflow (public Q&A for anyone who code...,Visit Stack Overflow;Go for a walk or other ph...,MacOS,"1,000 to 4,999 employees",iOS;Kubernetes;Linux;MacOS,iOS,I have little or no influence,NaN,Yes,"Yes, definitely",Less than once per month or monthly,Multiple times per day,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,True,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Objective-C;Python;Swift,Objective-C;Python;Swift,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Once a decade,NaN,NaN,No,NaN,NaN,NaN,Stack Overflow (public Q&A for anyone who codes),NaN,Linux-based,NaN,NaN,NaN,NaN,NaN,Yes,"Yes, somewhat",A few times per month or weekly,Daily or almost daily,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,True,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,ALL,NaN,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",NaN,White or of European descent,Man,Flex time or a flexible schedule;Office enviro...,Slightly dissatisfied,"I’m not actively looking, but I am open to new...",NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,Not at all important/not necessary,Curious about other opportunities;Wanting to w...,NaN,Once a year,Not sure,Yes,Yes,Occasionally: 1-2 days per quarter but less th...,NaN,NaN,Stack Overflow (public Q&A for anyone who code...,NaN,L

In [34]:
# sorting surveys results by country 
df.sort_values(by=['Country','SalaryUSD'],ascending=[True,False],inplace= True)

In [35]:
df[['Country','SalaryUSD']].head(50)

,Country,SalaryUSD
61540,Afghanistan,1000000.0
63925,Afghanistan,1000000.0
63993,Afghanistan,1000000.0
38261,Afghanistan,231192.0
28270,Afghanistan,130000.0
9806,Afghanistan,23328.0
62520,Afghanistan,23328.0
50812,Afghanistan,20950.0
50469,Afghanistan,20220.0
43642,Afghanistan,18660.0


In [36]:
df[['Country','SalaryUSD']].tail(50)

,Country,SalaryUSD
64206,NaN,NaN
64207,NaN,NaN
64210,NaN,NaN
64214,NaN,NaN
64233,NaN,NaN
64235,NaN,NaN
64245,NaN,NaN
64250,NaN,NaN
64251,NaN,NaN
64256,NaN,NaN


In [37]:
#get the 10 largest salaries 
df['SalaryUSD'].nlargest(10)

5701     2000000.0
53175    2000000.0
42657    2000000.0
58136    2000000.0
45753    2000000.0
121      2000000.0
123      2000000.0
191      2000000.0
663      2000000.0
697      2000000.0
Name: SalaryUSD, dtype: float64

In [38]:
df.nlargest(10,'SalaryUSD')

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,SalaryUSD,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
5701,5725,I am a developer by profession,False,26.0,14,Monthly,1.800000e+05,2000000.0,Canada,United States dollar,USD,Redis,NaN,"Developer, back-end;DevOps specialist","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,White or of European descent,Man,"Languages, frameworks, and other technologies ...",Slightly satisfied,I am not interested in new job opportunities,Go;Python;Ruby;TypeScript,Bash/Shell/PowerShell;JavaScript;Python,Teraform,NaN,"Github;Gitlab;Google Suite (Docs, Meet, etc)","Github;Google Suite (Docs, Meet, etc)",Yes,Extremely important,Very important,Trouble with my teammates;Better work/life bal...,"Read company media, such as employee blogs or ...",Once a year,No,Yes,No,Often: 1-2 days per week or more,NaN,"Hello, old friend",Stack Overflow (public Q&A for anyone who code...,Call a coworker or friend;Go for a walk or oth...,Linux-based,"10,000 or more employees",Docker;Google Cloud Platform,Docker;Google Cloud Platform,I have little or no influence,Straight / Heterosexual,Yes,"No, not at all",I have never participated in Q&A on Stack Over...,A few times per month or weekly,Easy,Too short,No,"Computer science, computer engineering, or sof...",Gatsby,Spring,Not applicable - I did not use Stack Overflow ...,40.0,10,4
53175,54263,I am a developer by profession,True,29.0,10,Weekly,7.000000e+04,2000000.0,Canada,United States dollar,USD,Redis,Firebase;PostgreSQL;SQLite,"Developer, mobile","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,South Asian,Man,"Flex time or a flexible schedule;Languages, fr...",Slightly satisfied,"I’m not actively looking, but I am open to new...",Go;Haskell;Rust,Dart;Java;JavaScript;Kotlin;Python;TypeScript,React Native;TensorFlow;Torch/PyTorch,Flutter;Node.js,Facebook Workplace,Confluence;Jira;Github;Gitlab;Slack;Microsoft ...,Yes,Extremely important,Somewhat important,Just because;Curious about other opportunities...,"Read company media, such as employee blogs or ...",Every few months,Not sure,Yes,Yes,Rarely: 1-2 days per year or less,Start a free trial;Ask developers I know/work ...,"Hello, old friend",Stack Overflow (public Q&A for anyone who code...,Meditate;Visit Stack Overflow;Go for a walk or...,MacOS,NaN,Docker;Heroku;iOS;Kubernetes,Android;AWS;Linux,I have some influence,Straight / Heterosexual,Not sure/can't remember,"Yes, somewhat",NaN,Daily or almost daily,Neither easy nor difficult,Too long,No,"Computer science, computer engineering, or sof...",React.js;Vue.js,Express;Flask;jQuery,Just as welcome now as I felt last year,36.0,14,5
42657,43430,I am a developer by profession,True,33.0,17,Monthly,1.710000e+05,2000000.0,Italy,United States dollar,USD,NaN,NaN,"Academic researcher;Developer, back-end;Develo...","Other doctoral degree (Ph.D., Ed.D., etc.)",Employed full-time,White or of European descent,Man,"Languages, frameworks, and other technologies ...",Very satisfied,I am not interested in new job opportunities,Assembly;Bash/Shell/PowerShell;C;C++;Perl;Python,Assembly;Bash/Shell/PowerShell;C;C++;Perl;Python,NaN,NaN,Confluence;Jira;Github;Gitlab;Slack;Google Sui...,Confluence;Jira;Github;Gitlab;Slack;Google Sui...,Yes,Somewhat important,Fairly important,Curious abou

In [39]:
df.nsmallest(500,'SalaryUSD')

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,SalaryUSD,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
35168,35295,I am a developer by profession,True,NaN,13,Monthly,2.0,0.0,Afghanistan,Afghan afghani,AFN,NaN,MySQL;PostgreSQL;SQLite,Academic researcher;Educator;Scientist,"Other doctoral degree (Ph.D., Ed.D., etc.)","Independent contractor, freelancer, or self-em...",NaN,NaN,NaN,Very dissatisfied,I am actively looking for a job,NaN,Assembly;Bash/Shell/PowerShell;C;C++;Java;Java...,NaN,TensorFlow;Unity 3D,NaN,NaN,NaN,NaN,Very important,NaN,NaN,Once a year,Yes,Onboarding? What onboarding?,Yes,Often: 1-2 days per week or more,Read ratings or reviews on third party sites l...,"Hello, old friend",Stack Overflow (public Q&A for anyone who code...,Play games;Go for a walk or other physical act...,Linux-based,"Just me - I am a freelancer, sole proprietor, ...",NaN,Android;iOS;Linux;MacOS,NaN,NaN,Yes,"No, not at all",Less than once per month or monthly,Multiple times per day,Neither easy nor difficult,Appropriate in length,NaN,"Another engineering discipline (such as civil,...",NaN,jQuery,A lot less welcome now than last year,100.0,35,27
46385,47357,I am a developer by profession,True,NaN,5,Weekly,1.0,0.0,Albania,Bhutanese ngultrum,BTN,MariaDB,NaN,Academic researcher,Primary/elementary school,Employed full-time,NaN,NaN,Family friendliness,Neither satisfied nor dissatisfied,I am actively looking for a job,Bash/Shell/PowerShell;C,NaN,Hadoop,NaN,"Google Suite (Docs, Meet, etc)",NaN,No,Extremely important,Somewhat important,Just because,"Read company media, such as employee blogs or ...",Once a decade,NaN,Yes,No,Rarely: 1-2 days per year or less,Start a free trial;Ask developers I know/work ...,"Hello, old friend",Stack Overflow (public Q&A for anyone who codes),Play games,Linux-based,"Just me - I am a freelancer, sole proprietor, ...",AWS,NaN,NaN,NaN,Yes,"No, not at all",Daily or almost daily,A few times per month or weekly,NaN,NaN,NaN,NaN,Drupal,NaN,Somewhat less welcome now than last year,168.0,2,11
6029,6054,I am a developer by profession,True,38.0,15,Yearly,0.0,0.0,Australia,Australian dollar,AUD,NaN,MySQL,"Designer;Developer, back-end;Developer, front-...","Secondary school (e.g. American high school, G...","Independent contractor, freelancer, or self-em...",NaN,Man,"Flex time or a flexible schedule;Languages, fr...",Very satisfied,I am not interested in new job opportunities,NaN,PHP,NaN,NaN,NaN,NaN,No,Neutral,Not at all important/not necessary,NaN,NaN,Once every few years,Not sure,Onboarding? What onboarding?,Yes,Often: 1-2 days per week or more,NaN,Indifferent,Stack Overflow (public Q&A for anyone who code...,NaN,MacOS,"Just me - I am a freelancer, sole proprietor, ...",NaN,MacOS,NaN,NaN,Yes,"Yes, definitely",Less than once per month or monthly,Multiple times per day,Easy,Too long,No,NaN,NaN,Laravel,A lot less welcome now than last year,30.0,22,20
27035,27135,I am a developer by profession,False,NaN,16,Yearly,0.0,0.0,Australia,Australian dollar,AUD,MySQL,MySQL,Senior executive/VP,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...",NaN,Man,Industry that I’d be working in;How widely use...,Very satisfied,"I’m not actively looking, but I am open to new...",Go;HTML/CSS;JavaScript;Python;SQL,HTML/CSS;JavaScr

#Grouping and Aggregating 

agrgregation: combining multiple pieces of data into a single result

mean,median, mode are aggregate operations

In [40]:
df['SalaryUSD'].head(15)

61540    1000000.0
63925    1000000.0
63993    1000000.0
38261     231192.0
28270     130000.0
9806       23328.0
62520      23328.0
50812      20950.0
50469      20220.0
43642      18660.0
15371      18000.0
2918       16335.0
26443      13992.0
56146      10884.0
26676       8016.0
Name: SalaryUSD, dtype: float64

In [41]:
df['SalaryUSD'].median()

54049.0

In [42]:
df.median()

Respondent     32231.0
Hobbyist           1.0
Age               29.0
CompTotal      63000.0
SalaryUSD      54049.0
WorkWeekHrs       40.0
dtype: float64

In [43]:
df.describe()

,Respondent,Age,CompTotal,SalaryUSD,WorkWeekHrs
count,64461.000000,45446.000000,3.482600e+04,3.475600e+04,41151.000000
mean,32554.079738,30.834111,3.190464e+242,1.037561e+05,40.782174
std,18967.442360,9.585392,inf,2.268853e+05,17.816383
min,1.000000,1.000000,0.000000e+00,0.000000e+00,1.000000
25%,16116.000000,24.000000,2.000000e+04,2.464800e+04,40.000000
50%,32231.000000,29.000000,6.300000e+04,5.404900e+04,40.000000
75%,49142.000000,35.000000,1.250000e+05,9.500000e+04,44.000000
max,65639.000000,279.000000,1.111111e+247,2.000000e+06,475.000000


In [44]:
#the mean is affected heavily by outliers 

In [45]:
df['SalaryUSD'].count()

34756

In [46]:
df['Hobbyist'].value_counts(normalize=True)

True     0.782228
False    0.217772
Name: Hobbyist, dtype: float64

In [47]:
df['Country'].value_counts()

United States     12469
India              8403
United Kingdom     3896
Germany            3890
Canada             2191
                  ...  
Lesotho               1
Chad                  1
Kiribati              1
Saint Lucia           1
Liechtenstein         1
Name: Country, Length: 183, dtype: int64

In [48]:
country_grp = df.groupby(['Country'])

In [49]:
country_grp.get_group('India')

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,SalaryUSD,Country,CurrencyDesc,CurrencySymbol,DatabaseDesireNextYear,DatabaseWorkedWith,DevType,EdLevel,Employment,Ethnicity,Gender,JobFactors,JobSat,JobSeek,LanguageDesireNextYear,LanguageWorkedWith,MiscTechDesireNextYear,MiscTechWorkedWith,NEWCollabToolsDesireNextYear,NEWCollabToolsWorkedWith,NEWDevOps,NEWDevOpsImpt,NEWEdImpt,NEWJobHunt,NEWJobHuntResearch,NEWLearn,NEWOffTopic,NEWOnboardGood,NEWOtherComms,NEWOvertime,NEWPurchaseResearch,NEWPurpleLink,NEWSOSites,NEWStuck,OpSys,OrgSize,PlatformDesireNextYear,PlatformWorkedWith,PurchaseWhat,Sexuality,SOAccount,SOComm,SOPartFreq,SOVisitFreq,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
37333,37655,I am a developer by profession,False,NaN,15,Monthly,1.500000e+05,1800000.0,India,United States dollar,USD,Firebase,Firebase;Microsoft SQL Server,"Developer, front-end;Developer, full-stack;Dev...","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,NaN,Slightly satisfied,NaN,Dart;Objective-C;Python,C#;Dart;Java;Objective-C;Python;SQL;Swift,Flutter;TensorFlow,.NET;Flutter;Node.js;React Native;TensorFlow;X...,Confluence;Jira;Github;Microsoft Teams,Confluence;Jira;Github;Microsoft Teams;Trello;...,Yes,Neutral,Very important,NaN,NaN,Every few months,NaN,Yes,NaN,Sometimes: 1-2 days per month but less than we...,NaN,Amused,NaN,Play games;Visit Stack Overflow;Go for a walk ...,MacOS,500 to 999 employees,Android;iOS;Raspberry Pi,Android;Google Cloud Platform;iOS;Raspberry Pi...,I have little or no influence,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",Express,ASP.NET;Express,NaN,40.0,13,10
36726,36976,"I am not primarily a developer, but I write co...",False,NaN,20,Yearly,1.600000e+06,1600000.0,India,United States dollar,USD,NaN,NaN,Data or business analyst;Database administrato...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Employed full-time,NaN,NaN,NaN,Very satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Neutral,Critically important,NaN,NaN,NaN,NaN,Yes,NaN,Often: 1-2 days per week or more,NaN,NaN,NaN,NaN,NaN,100 to 499 employees,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,NaN,50.0,15,10
3318,3332,I am a developer by profession,True,NaN,19,Monthly,4.200000e+07,1000000.0,India,Indian rupee,INR,Cassandra;DynamoDB;Elasticsearch;MongoDB;MySQL...,MongoDB;MySQL;PostgreSQL;Redis,"Developer, back-end","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed full-time,NaN,NaN,Flex time or a flexible schedule;Specific depa...,Neither satisfied nor dissatisfied,"I’m not actively looking, but I am open to new...",Go;Java;JavaScript;Python;Scala;SQL,Go;Java;Python;Scala;SQL,Apache Spark;Node.js;Pandas;TensorFlow;Teraform,Ansible;Apache Spark;Node.js;Pandas;TensorFlow,Github;Slack;Microsoft Azure;Trello,Confluence;Jira;Github;Slack;Microsoft Azure;T...,No,Neutral,Fairly important,Just because;Better compensation;Trouble with ...,Company reviews from third party sites (e.g. G...,Every few months,No,No,Yes,Sometimes: 1-2 days per month but less than we...,Start a free trial;Ask developers I know/work ...,Indifferent,Stack Overflow (public Q&A for anyone who code...,Call a coworker or friend;Visit Stack Overflow...,MacOS,"10,000 or more employees",AWS;Docker;Google Cloud Platform;Heroku;Kubern...,AWS;Docker;Kubernetes;Linux;MacOS;Microsoft Azure,I have some influence,NaN,Yes,"Yes, definitely",Daily or almost daily,Daily or almost daily,Neither easy nor difficult,Appropriate in length,NaN,"Computer science, computer engineering, or sof...",Django;Flask;Spring,Flask;Spring,Somewhat less welcome now than last year,35.0,7,3
8425,8457,I am a developer by profession,True,22.0,18,Monthly,7.000000e+06,1000000.0,India,Indian rupee,INR,Elasticsearch;MySQL;Oracle,MySQL;Oracle,Academic researcher;Data scientist or machine ...,"Bachelor’s degree (B.A.,

In [50]:
filt = df['Country']== 'India'
df.loc[filt]['SalaryUSD'].value_counts()

5028.0      66
16754.0     62
8377.0      55
8376.0      55
2508.0      53
            ..
256332.0     1
8524.0       1
42583.0      1
21292.0      1
419.0        1
Name: SalaryUSD, Length: 620, dtype: int64

In [51]:
country_grp['SalaryUSD'].value_counts().head(50)

Country      SalaryUSD
Afghanistan  1000000.0    3
             23328.0      2
             0.0          1
             109.0        1
             780.0        1
             1860.0       1
             2328.0       1
             4356.0       1
             4509.0       1
             4668.0       1
             5448.0       1
             8016.0       1
             10884.0      1
             13992.0      1
             16335.0      1
             18000.0      1
             18660.0      1
             20220.0      1
             20950.0      1
             130000.0     1
             231192.0     1
Albania      15900.0      3
             25944.0      2
             31788.0      2
             0.0          1
             2124.0       1
             4344.0       1
             5832.0       1
             10596.0      1
             10812.0      1
             12972.0      1
             13776.0      1
             16860.0      1
             21192.0      1
             23316.0     

In [52]:
country_grp['SalaryUSD'].value_counts().loc['India']

SalaryUSD
5028.0       66
16754.0      62
8376.0       55
8377.0       55
2508.0       53
             ..
686916.0      1
698086.0      1
871212.0      1
1600000.0     1
1800000.0     1
Name: SalaryUSD, Length: 620, dtype: int64

In [53]:
country_grp['SalaryUSD'].median().loc['Germany']

62697.0

In [54]:
country_grp['SalaryUSD'].agg(['median','mean']).loc['Canada']

median     68068.000000
mean      115101.775465
Name: Canada, dtype: float64

In [55]:
filt = df['Country']== 'India'
df.loc[filt]['LanguageWorkedWith'].str.contains('Python').sum()

2670

In [56]:
country_grp['LanguageWorkedWith'].apply(lambda x: x.str.contains('Python').sum())

Country
Afghanistan                              11
Albania                                  13
Algeria                                  40
Andorra                                   3
Angola                                    1
                                       ... 
Venezuela, Bolivarian Republic of...     29
Viet Nam                                102
Yemen                                     1
Zambia                                    4
Zimbabwe                                 13
Name: LanguageWorkedWith, Length: 183, dtype: int64

In [57]:
country_respondents= df['Country'].value_counts()
country_respondents
#this is the total people from each country who responded to the survey

United States     12469
India              8403
United Kingdom     3896
Germany            3890
Canada             2191
                  ...  
Lesotho               1
Chad                  1
Kiribati              1
Saint Lucia           1
Liechtenstein         1
Name: Country, Length: 183, dtype: int64

In [58]:
country_uses_python =country_grp['LanguageWorkedWith'].apply(lambda x: x.str.contains('Python').sum())
country_uses_python
#this is the number of people from each country who said that that they use python


Country
Afghanistan                              11
Albania                                  13
Algeria                                  40
Andorra                                   3
Angola                                    1
                                       ... 
Venezuela, Bolivarian Republic of...     29
Viet Nam                                102
Yemen                                     1
Zambia                                    4
Zimbabwe                                 13
Name: LanguageWorkedWith, Length: 183, dtype: int64

In [59]:
python_df = pd.concat([country_respondents,country_uses_python], axis='columns',sort=False) 
python_df

,Country,LanguageWorkedWith
United States,12469,5964
India,8403,2670
United Kingdom,3896,1621
Germany,3890,1712
Canada,2191,1011
...,...,...
Lesotho,1,0
Chad,1,0
Kiribati,1,0
Saint Lucia,1,1


In [60]:
python_df.rename(columns={'Country':'NumRespondents','LanguageWorkedWith':'NumKnowsPython'},inplace=True)

In [61]:
python_df

,NumRespondents,NumKnowsPython
United States,12469,5964
India,8403,2670
United Kingdom,3896,1621
Germany,3890,1712
Canada,2191,1011
...,...,...
Lesotho,1,0
Chad,1,0
Kiribati,1,0
Saint Lucia,1,1


In [62]:
python_df['PctKnowsPython']= (python_df['NumKnowsPython']/python_df['NumRespondents'])*100
python_df

,NumRespondents,NumKnowsPython,PctKnowsPython
United States,12469,5964,47.830620
India,8403,2670,31.774366
United Kingdom,3896,1621,41.606776
Germany,3890,1712,44.010283
Canada,2191,1011,46.143314
...,...,...,...
Lesotho,1,0,0.000000
Chad,1,0,0.000000
Kiribati,1,0,0.000000
Saint Lucia,1,1,100.000000


In [63]:
python_df.sort_values(by='PctKnowsPython',ascending=False)

,NumRespondents,NumKnowsPython,PctKnowsPython
Gabon,1,1,100.000000
Saint Lucia,1,1,100.000000
"Micronesia, Federated States of...",1,1,100.000000
Montenegro,13,9,69.230769
Brunei Darussalam,3,2,66.666667
...,...,...,...
Djibouti,2,0,0.000000
Gambia,4,0,0.000000
Monaco,2,0,0.000000
Cape Verde,2,0,0.000000


In [64]:
python_df.head(50)

,NumRespondents,NumKnowsPython,PctKnowsPython
United States,12469,5964,47.830620
India,8403,2670,31.774366
United Kingdom,3896,1621,41.606776
Germany,3890,1712,44.010283
Canada,2191,1011,46.143314
France,1898,833,43.888303
Brazil,1818,635,34.928493
Netherlands,1343,544,40.506329
Poland,1278,509,39.827856
Australia,1208,524,43.377483


In [65]:
python_df.loc['Japan']

NumRespondents    297.000000
NumKnowsPython    132.000000
PctKnowsPython     44.444444
Name: Japan, dtype: float64

#Data cleaning

In [74]:
df['YearsCode'].unique()

array(['36', '7', '4', '15', '6', '17', '8', '10', '35', '5', '37', '19',
       '9', '22', '30', '23', '20', '2', 'Less than 1 year', '3', '13',
       '25', '16', '43', '11', '38', '33', nan, '24', '21', '12', '40',
       '27', '50', '46', '14', '18', '28', '32', '44', '26', '42', '31',
       '34', '29', '1', '39', '41', '45', 'More than 50 years', '47',
       '49', '48'], dtype=object)

In [75]:
df['YearsCode'].replace('Less than 1 year',0,inplace=True)

In [77]:
df['YearsCode'].replace('More than 50 years',51,inplace=True)

In [78]:
df['YearsCode'].unique()

array(['36', '7', '4', '15', '6', '17', '8', '10', '35', '5', '37', '19',
       '9', '22', '30', '23', '20', '2', 0, '3', '13', '25', '16', '43',
       '11', '38', '33', nan, '24', '21', '12', '40', '27', '50', '46',
       '14', '18', '28', '32', '44', '26', '42', '31', '34', '29', '1',
       '39', '41', '45', 51, '47', '49', '48'], dtype=object)

In [83]:
df['YearsCode'] = df['YearsCode'].astype(float)

In [84]:
df['YearsCode'].mean()

12.709052770265584

In [85]:
df['YearsCode'].median()

10.0

the average of coding experience is 12 years while the median is 9 years. 